In [1]:
!pip install yfinance pandas numpy matplotlib ta mplfinance

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/949.2 kB ? eta -:--:--
     ---------------------------------------- 949.2/949.2 kB 8.8 MB/s  0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Gettin

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\moonj\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
!python.exe -m pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 25.0.1
    Uninstalling pip-25.0.1:
      Successfully uninstalled pip-25.0.1


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [19]:
%pip install numpy pandas yfinance matplotlib mplfinance ta pandas_ta

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --------------------------------- ------ 2.4/2.8 MB 12.2 MB/s eta 0:00:01
   ---------------------------------------- 2.8/2.8 MB 11.0 MB/s  0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ------- -------------------------------- 2.4/12.6 MB 13.4 MB/s eta 0:00:01
   ---------------- ----------------------- 5.2/12.6 MB 12.7 MB/s eta 0:00:01
   -------------------- ------------------- 6.6/12.6 MB 11.2 MB/s eta 0:00:01
   --------------------------- ------------ 8.7/12.6 MB 10.3 MB/s eta 0:00:01
   ----------------------------------- ---- 11.3/12.6 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 10.5 MB/s  0:00:01
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   --- ------------------------------------ 2.6/30.3 MB 12.5 MB/s eta 0:00:03
   ------ ------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.


In [30]:
%pip install yfinance pandas numpy matplotlib mplfinance pandas_ta

  Using cached numba-0.61.2-cp312-cp312-win_amd64.whl.metadata (2.9 kB)
Using cached numba-0.61.2-cp312-cp312-win_amd64.whl (2.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [34]:
# =========================
# QLD 차트: MA + PSAR + 회귀채널 (pandas_ta 사용, numba 불필요)
# =========================
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import mplfinance as mpf
import pandas_ta as pta
from datetime import datetime

# ---------- 파라미터 ----------
TICKER = "QLD"        # 미국 ProShares Ultra QQQ: "QLD" / 호주 상장: "QLD.AX"
PERIOD = "10y"        # 1y, 5y, 10y, max ...
INTERVAL = "1d"       # 1d, 1wk, 1mo
MA_WINDOWS = [5, 20, 60, 120]
PSAR_STEP = 0.02
PSAR_MAX_STEP = 0.2
REG_CHANNEL_WINDOW = 120
REG_CHANNEL_K = 2.0

# ---------- 데이터 가져오기 ----------
raw = yf.download(TICKER, period=PERIOD, interval=INTERVAL, auto_adjust=False)
if raw.empty:
    raise SystemExit("데이터가 비었습니다. TICKER/기간/인터벌을 확인하세요.")
df = raw.copy()

# dtype/NaN 정리(안전)
for col in ["High", "Low", "Close", "Open"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=["High","Low","Close"]).copy()

# yfinance 분할 정보(있으면 사용)
splits = df["Stock Splits"] if "Stock Splits" in df.columns else pd.Series(dtype=float)

# ---------- 이동평균선 ----------
for w in MA_WINDOWS:
    df[f"MA{w}"] = df["Close"].rolling(w).mean()

# ---------- PSAR (pandas_ta) ----------
# pandas_ta는 af(가속도)와 max_af를 받으며 PSAR, PSARl(롱), PSARs(숏) 컬럼을 반환
psar_df = pta.psar(
    high=df["High"], low=df["Low"], close=df["Close"],
    af=PSAR_STEP, max_af=PSAR_MAX_STEP
)

# 컬럼명이 버전에 따라 접미사가 붙을 수 있어 안전하게 매핑
def _pick_col(cols, prefix):
    cands = [c for c in cols if c.upper().startswith(prefix.upper())]
    return cands[0] if cands else None

col_psar  = _pick_col(psar_df.columns, "PSAR")
col_psarl = _pick_col(psar_df.columns, "PSARL")
col_psars = _pick_col(psar_df.columns, "PSARS")

df["PSAR"] = psar_df[col_psar] if col_psar else np.nan
df["PSAR_up"] = psar_df[col_psarl] if col_psarl else np.nan
df["PSAR_down"] = psar_df[col_psars] if col_psars else np.nan

# 추세 플래그: 롱 값 존재→상승, 숏 값 존재→하락
df["PSAR_trend_up"] = df["PSAR_up"].notna().astype(int)
df["PSAR_trend_down"] = df["PSAR_down"].notna().astype(int)

# 강세/약세 전환 포인트 (down->up / up->down)
df["bull_flip"] = (df["PSAR_trend_up"].shift(1) == 0) & (df["PSAR_trend_up"] == 1)
df["bear_flip"] = (df["PSAR_trend_down"].shift(1) == 0) & (df["PSAR_trend_down"] == 1)

# ---------- 회귀 채널 (최근 윈도우 1차 회귀 + 잔차 표준편차) ----------
x = np.arange(len(df))
y = df["Close"].values
fit_line = np.full_like(y, np.nan, dtype=float)
upper = np.full_like(y, np.nan, dtype=float)
lower = np.full_like(y, np.nan, dtype=float)

for i in range(REG_CHANNEL_WINDOW - 1, len(df)):
    xs = x[i-REG_CHANNEL_WINDOW+1:i+1]
    ys = y[i-REG_CHANNEL_WINDOW+1:i+1]
    a, b = np.polyfit(xs, ys, 1)  # y = a*x + b
    y_hat = a*xs + b
    resid = ys - y_hat
    s = resid.std(ddof=1)
    fit_line[i] = a*x[i] + b
    upper[i] = fit_line[i] + REG_CHANNEL_K*s
    lower[i] = fit_line[i] - REG_CHANNEL_K*s

df["reg_mid"] = fit_line
df["reg_up"] = upper
df["reg_dn"] = lower

# ---------- 최고/최저 주석 ----------
low_idx = df["Close"].idxmin()
high_idx = df["Close"].idxmax()
low_val = df.loc[low_idx, "Close"]
high_val = df.loc[high_idx, "Close"]

# ---------- 월 배경 음영 ----------
def month_spans(ax, index):
    if len(index) == 0:
        return
    months = pd.Series(index).to_series().dt.to_period("M")
    groups = months.groupby(months).groups
    toggle = False
    for _, idxs in groups.items():
        start = index[min(idxs)]
        end = index[max(idxs)]
        if toggle:
            ax.axvspan(start, end, alpha=0.07)
        toggle = not toggle

# ---------- addplot 구성 ----------
apds = []
# 이동평균선
for w in MA_WINDOWS:
    apds.append(mpf.make_addplot(df[f"MA{w}"], panel=0))
# PSAR 점 찍기
apds.append(mpf.make_addplot(df["PSAR"], scatter=True, markersize=10, panel=0))
# 회귀 채널
apds.append(mpf.make_addplot(df["reg_mid"], panel=0))
apds.append(mpf.make_addplot(df["reg_up"], panel=0))
apds.append(mpf.make_addplot(df["reg_dn"], panel=0))
# 강/약세 전환 화살표
bull_y = df.loc[df["bull_flip"], "Low"] * 0.995
bear_y = df.loc[df["bear_flip"], "High"] * 1.005
apds.append(mpf.make_addplot(bull_y, type="scatter", markersize=80, marker="^", panel=0))
apds.append(mpf.make_addplot(bear_y, type="scatter", markersize=80, marker="v", panel=0))

# ---------- 차트 그리기 ----------
fig, axlist = mpf.plot(
    df,
    type="candle",
    volume=True,
    addplot=apds,
    figratio=(21, 7),
    figscale=1.2,
    title=f"{TICKER} with MAs, PSAR (pandas_ta), Regression Channel",
    style="yahoo",
    returnfig=True
)
ax = axlist[0]

# 월별 배경 음영
month_spans(ax, df.index)

# 분할 주석
if not splits.empty:
    split_points = splits[splits != 0.0]
    for dt, ratio in split_points.items():
        ax.annotate(
            f"액면분할({ratio:+.2f}x)",
            xy=(dt, df.loc[dt, "Close"]),
            xytext=(0, 30),
            textcoords="offset points",
            arrowprops=dict(arrowstyle="->", lw=1),
            ha="center", va="bottom", fontsize=8
        )

# 최고/최저 주석
ax.annotate(
    f"최저 {low_val:.4f} ({low_idx:%Y/%m})",
    xy=(low_idx, low_val),
    xytext=(-40, -40),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->", lw=1),
    fontsize=9
)
ax.annotate(
    f"최고 {high_val:.4f} ({high_idx:%Y/%m})",
    xy=(high_idx, high_val),
    xytext=(20, 40),
    textcoords="offset points",
    arrowprops=dict(arrowstyle="->", lw=1),
    fontsize=9
)

plt.tight_layout()
plt.show()


ImportError: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

In [35]:
%pip install --upgrade git+https://github.com/twopirllc/pandas-ta.git

  Cloning https://github.com/twopirllc/pandas-ta.git to c:\users\moonj\appdata\local\temp\pip-req-build-jm5c577u
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/twopirllc/pandas-ta.git 'C:\Users\moonj\AppData\Local\Temp\pip-req-build-jm5c577u'
  bash: line 1: /dev/tty: No such device or address
  error: failed to execute prompt script (exit code 1)
  fatal: could not read Username for 'https://github.com': No such file or directory
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/twopirllc/pandas-ta.git 'C:\Users\moonj\AppData\Local\Temp\pip-req-build-jm5c577u' did not run successfully.
  │ exit code: 128
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
ERROR: Failed to build 'git+https://github.com/twopirllc/pandas-ta.git' when git clone --filter=blob:none --quiet https://github.com/twopirllc/pandas-ta.git 'c:\users\moonj\appdata\local\temp\pip-req-build-jm5c577u'


In [32]:
%pip install --upgrade --force-reinstall "numpy<2.3"

  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# =========================
# Candles + MAs + Regression Channel + Monthly Bands
# =========================
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import mplfinance as mpf

# ---------- 설정 ----------
TICKER = "QLD"          # 예: "QLD", 호주면 "QLD.AX"
PERIOD = "10y"          # 1y, 5y, 10y, max
INTERVAL = "1d"         # 1d, 1wk, 1mo
MA_WINDOWS = [5, 20, 60, 120]

# 회귀 채널 설정 (최근 윈도우로 1차 회귀 + 잔차표준편차 * k)
REG_WINDOW = 120
REG_K = 2.0

# 5-20 골든/데드 크로스 화살표 표시 여부
SHOW_MA_CROSS_ARROWS = True

# ---------- 데이터 다운로드 ----------
df = yf.download(
    TICKER,
    period=PERIOD,
    interval=INTERVAL,
    auto_adjust=False,
    group_by='ticker'
)

if df.empty:
    raise SystemExit("⚠️ 데이터가 비었습니다. TICKER/기간/인터벌을 확인하세요.")

# 멀티인덱스 컬럼(여러 티커 동시 다운로드) 대비
# 단일 티커면 일반 단일 컬럼으로 나옵거나, 멀티인덱스여도 level=0이 티커입니다.
if isinstance(df.columns, pd.MultiIndex):
    df = df.xs(TICKER, axis=1, level=0)

# 형 변환/결측 처리
num_cols = [c for c in ["Open", "High", "Low", "Close", "Adj Close", "Volume"] if c in df.columns]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")
df = df.dropna(subset=[c for c in ["High", "Low", "Close"] if c in df.columns]).copy()

# yfinance의 Stock Splits(있으면 사용)
splits = df["Stock Splits"] if "Stock Splits" in df.columns else pd.Series(dtype=float)

# ---------- 이동평균 ----------
for w in MA_WINDOWS:
    df[f"MA{w}"] = df["Close"].rolling(w).mean()

# ---------- 회귀 채널(윈도우 고정) ----------
x = np.arange(len(df))  # 0..N-1
y = df["Close"].values

mid = np.full_like(y, np.nan, dtype=float)
up = np.full_like(y, np.nan, dtype=float)
dn = np.full_like(y, np.nan, dtype=float)

for i in range(REG_WINDOW - 1, len(df)):
    xs = x[i-REG_WINDOW+1:i+1]
    ys = y[i-REG_WINDOW+1:i+1]
    a, b = np.polyfit(xs, ys, 1)        # y = a*x + b
    yhat = a*xs + b
    s = (ys - yhat).std(ddof=1)
    mid[i] = a*x[i] + b
    up[i]  = mid[i] + REG_K*s
    dn[i]  = mid[i] - REG_K*s

df["reg_mid"] = mid
df["reg_up"]  = up
df["reg_dn"]  = dn

# ---------- MA 크로스(5 vs 20) 화살표 ----------
if SHOW_MA_CROSS_ARROWS and all(f"MA{w}" in df.columns for w in (5, 20)):
    ma5 = df["MA5"]
    ma20 = df["MA20"]
    golden = (ma5.shift(1) <= ma20.shift(1)) & (ma5 > ma20)   # 상향 교차
    dead   = (ma5.shift(1) >= ma20.shift(1)) & (ma5 < ma20)   # 하향 교차
    bull_y = df.loc[golden, "Low"] * 0.995
    bear_y = df.loc[dead, "High"] * 1.005
else:
    bull_y = pd.Series(dtype=float)
    bear_y = pd.Series(dtype=float)

# ---------- 월 교차 음영 함수 ----------
def shade_months(ax, index):
    if len(index) == 0:
        return
    months = pd.Series(index).to_series().dt.to_period("M")
    groups = months.groupby(months).groups
    toggle = False
    for _, idxs in groups.items():
        start = index[min(idxs)]
        end = index[max(idxs)]
        ax.axvspan(start, end, alpha=0.07, facecolor=("tab:red" if toggle else "tab:blue"))
        toggle = not toggle

# ---------- 부가 플롯 구성 ----------
apds = []
for w in MA_WINDOWS:
    apds.append(mpf.make_addplot(df[f"MA{w}"], panel=0))
apds += [
    mpf.make_addplot(df["reg_mid"], panel=0),
    mpf.make_addplot(df["reg_up"], panel=0),
    mpf.make_addplot(df["reg_dn"], panel=0),
]
# 화살표(상향 ^, 하향 v)
if not bull_y.empty:
    apds.append(mpf.make_addplot(bull_y, type="scatter", marker="^", markersize=80, panel=0))
if not bear_y.empty:
    apds.append(mpf.make_addplot(bear_y, type="scatter", marker="v", markersize=80, panel=0))

# ---------- 차트 렌더 ----------
fig, axlist = mpf.plot(
    df,
    type="candle",
    volume=True,
    addplot=apds,
    figratio=(21, 7),
    figscale=1.2,
    title=f"{TICKER} • MAs + Regression Channel",
    style="yahoo",
    returnfig=True
)
ax = axlist[0]

# 월별 음영
shade_months(ax, df.index)

# 최저/최고 주석
low_idx = df["Close"].idxmin(); low_val = df.loc[low_idx, "Close"]
high_idx = df["Close"].idxmax(); high_val = df.loc[high_idx, "Close"]
ax.annotate(f"최저 {low_val:.4f} ({low_idx:%Y/%m})",
            xy=(low_idx, low_val), xytext=(-40, -40),
            textcoords="offset points", arrowprops=dict(arrowstyle="->", lw=1),
            fontsize=9)
ax.annotate(f"최고 {high_val:.4f} ({high_idx:%Y/%m})",
            xy=(high_idx, high_val), xytext=(20, 40),
            textcoords="offset points", arrowprops=dict(arrowstyle="->", lw=1),
            fontsize=9, color="red")

# 액면분할 주석(있을 때만)
if not splits.empty:
    split_points = splits[splits != 0.0]
    for dt, ratio in split_points.items():
        ax.annotate(f"액면분할({ratio:+.2f}x)",
                    xy=(dt, df.loc[dt, "Close"]),
                    xytext=(0, 25), textcoords="offset points",
                    arrowprops=dict(arrowstyle="->", lw=1),
                    ha="center", va="bottom", fontsize=8)

plt.tight_layout()
plt.show()


[*********************100%***********************]  1 of 1 completed


KeyError: 'QLD'

In [40]:
# =========================
# QLD 차트: 이동평균 + 회귀채널 (PSAR 없음)
# =========================
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime

# ---------- 파라미터 ----------
TICKER = "QLD"        # 미국: "QLD" / 호주 상장: "QLD.AX"
PERIOD = "10y"        # 1y, 5y, 10y, max ...
INTERVAL = "1d"       # 1d, 1wk, 1mo
MA_WINDOWS = [5, 20, 60, 120]
REG_CHANNEL_WINDOW = 120
REG_CHANNEL_K = 2.0

# ---------- 데이터 가져오기 ----------
df = yf.download(TICKER, period=PERIOD, interval=INTERVAL, auto_adjust=False)



[*********************100%***********************]  1 of 1 completed


In [41]:
df.head()

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,QLD,QLD,QLD,QLD,QLD,QLD
Date,,,,,,
2015-10-28,10.030323,10.18750,10.18750,9.8950,10.06000,14408800
2015-10-29,9.999557,10.15625,10.18875,10.0725,10.08875,8737600
2015-10-30,9.912173,10.06750,10.19875,10.0625,10.16250,10600800
2015-11-02,10.138631,10.29750,10.31250,10.0750,10.12125,8164800
2015-11-03,10.206315,10.36625,10.43250,10.2200,10.25000,6733600
